In [1]:
## Importing Libraries

# Python-based Libraries


# Data Analysis Libraries
import pandas as pd

# Graph plotting libraries

    
# Filtering Data before Plotting


In [22]:
from datetime import timedelta


class Drifters:
    
    def __init__(self):
        self.data = {}
        self.logsheet = pd.DataFrame()
        
    def read_data(self):
        pass
    
    def read_logsheet(self):
        pass
    
    def time_shift(self, shift_amount:float = 1):
        for key in self.data.keys():
            self.data[key]["DateTime"] = self.data[key]["DateTime"] + timedelta(hours = shift_amount)
    
    def extract_data(self):
        pass
    
    def compute_velocity(self):
        pass
    
    def plot_trajectory(self):
        pass
    
        
    

In [23]:
# Testing code
dftr = Drifters()
dftr.data[119] = pd.DataFrame({"DateTime": ["11-11-2022 9:00:00", "11-11-2022 10:00:00", "11-11-2022 12:00:00"], 
                               "Long": [5.9, 6.0, 6.1], "Lat": [43.5, 43.6, 43.7]})

dftr.data[119]["DateTime"] = pd.to_datetime(dftr.data[119]["DateTime"])
dftr.data[119].head()

,DateTime,Long,Lat
0,2022-11-11 09:00:00,5.9,43.5
1,2022-11-11 10:00:00,6.0,43.6
2,2022-11-11 12:00:00,6.1,43.7


In [24]:
dftr.time_shift(shift_amount=-2)
dftr.data[119].head()

,DateTime,Long,Lat
0,2022-11-11 07:00:00,5.9,43.5
1,2022-11-11 08:00:00,6.0,43.6
2,2022-11-11 10:00:00,6.1,43.7
